# Multi-Gaussian example

As an introductory example we can use astroABC to find the posterior distribution for some Gaussian distributed data. Although in this case we already know the likelihood this example is to illustrate how to call astroABC and provide user-defined metrics.


In [45]:
# start by importing astroabc and numpy
import numpy as np
import astroabc

We need to provide:

- a dataset
- a forwards simulating model for the data
- a method defining the distance metric.

For this example we generate a dataset of a 1000 draws from a 2D multi-Gaussian where the true means are e.g

$\mu_{1,true} = 0.037579, \, \mu_{2, true}=0.573537$

and we have fixed the covariance matrix to be  a diagonal matrix with $\sigma_1^2 = \sigma_2^2 = 0.05$. 
We can do this using an inbuilt model class in astroabc.

In [46]:
#make the fake data with diagonal covariance
means= np.array([0.037579, 0.573537])
cov = np.array([0.02,0.1])
data = astroabc.Model("normal",1000).make_mock(means,cov)

In this example the make_mock method also provides a forwards simulating model for the data. 

In [47]:
#define a method for simulating the data given input parameters
model_sim = astroabc.Model("normal",10000).make_mock

Next we define a distance metric method. In this example instead of using all of the data (all 1000 draws from a 2D Gaussian) we use the means of the data as a summary statistic and our distance metric is the sum over the difference in the means for the 2D Gaussian 

In [48]:
def dist_metric(d,x):
    return np.sum(np.abs(np.mean(x,axis=0) - np.mean(d,axis=0)))

Next we specify priors for each of the parameters we want to vary in the sampler. This is done by specifying a list of tuples. The zeroth element in each tuple should be a string specifying the prior for this parameter and the first element should be a list of the hyperparameters needed for this prior.
e.g. below we use a normal distribution with mean  0 and variance 0.5 for the first parameter and a uniform distribution from 0.1 - 0.9 for the second parameter.

In [49]:
priors =  [('normal', [0.0,0.1]), ('uniform', [0.1, 0.9])]

Next we need to set some keywords for astroABC. This can be done by creating a dictionary of inputs which are passed to the sampler. Many of these entries have defaults and do not need to be specified explicitly.
Only the name of the distance metric method needs to be explicity provided as a keyword.
The full set of keywords are given in the doc string of the class. Some examples are

- tol_type: which specifies the decreasing tolerance levels. "exp","lin", "log" and "const" are options. (default = 'exp')

- verbose: level of verbosity, 0 = no printout to screen, 1 = print to screen  (default = 0)

- adapt_t: Boolean True/False for adaptive threshold setting (default = False)

- threshold: qth quantile used in adaptive threshold setting (default = 75)

- pert_kernel: 0 =weighted covariance, 1= Filippi, 2 = TVZ, 3= Leodoit_Wolf (default = 0)

- dfunc:method for calculating the distance metric

- from_restart: Boolean True/False

- restart: string name of restart file

- outfile:string specifying name of output file (default = abc_out.txt)

- mpi: Boolean True/False (default = False)

- mp:Boolean True/False (default = False)

- num_proc:number of threads for mp setting (default = None)

Please see the doc strings of the astroABC sampler for details on each of these settings.

In [50]:
prop={'dfunc':dist_metric, 'outfile':"gaussian_example.txt", 'verbose':1, 'adapt_t': True}

Now we are ready to create an instance of our sampler. 
To do this we just need to specify the following to the astroabc.ABC_class

astroabc.ABC_class(number of parameters,number of particles,data,tolerance levels,number of iterations,priors,prop)

To begin let's run in serial using 100 particles for 30 iterations with default tolerance levels of a maximum threshold of 0.7 and  a minimum threshold of 0.05:

In [51]:
sampler = astroabc.ABC_class(2,100,data,[0.5,0.002],25,priors,**prop)

	 	
	 ########################     astroABC     ########################	
	 	
	 Npart=100 	 numt=25 	 tol=[0.5000,0.0020] exp
	 Priors= [('normal', [0.0, 0.1]), ('uniform', [0.1, 0.9])]


Then we simply begin sampling on our data...

In [52]:
sampler.sample(model_sim)

	 Running sampler...	 
	 Step: 0 	 tol: 0.5 	 Params: [-0.00016293735448206571, 0.52217979293248551]
	 Step: 1 	 tol: 0.353814428765 	 Params: [0.048264109132819186, 0.5563569337505947]
	 Step: 2 	 tol: 0.276186464038 	 Params: [0.025584974366035639, 0.58070754217795828]
	 Step: 3 	 tol: 0.216363674973 	 Params: [0.030255163959473478, 0.58794187987138047]
	 Step: 4 	 tol: 0.190889551422 	 Params: [0.040277666777101935, 0.56393487225279815]
	 Step: 5 	 tol: 0.153359562973 	 Params: [0.039306286160370185, 0.56851522343578831]
	 Step: 6 	 tol: 0.128463146703 	 Params: [0.043121346205092781, 0.57729286761401999]
	 Step: 7 	 tol: 0.111204377817 	 Params: [0.040257342462254188, 0.56809937171859981]
	 Step: 8 	 tol: 0.0904448796102 	 Params: [0.043475785485935677, 0.57011869709747265]
	 Step: 9 	 tol: 0.0789762626241 	 Params: [0.034331593552310594, 0.56893960364865104]
	 Step: 10 	 tol: 0.0693245861078 	 Params: [0.03700345377208468, 0.57377512562268473]
	 Step: 11 	 tol: 0.0583577860932 	 P

The output above shows the estimated means of the 2D Gaussian averaged over all 100 particles at each iteration, together with the tolerance level. Note above that the sampling may end before 20 iterations if the minimum tolerance level is reached first.
Recall that the true parameter values are $\mu_{1,true} = 0.037579, \, \mu_{2, true}=0.573537$

# K-Nearest Neighbours estimation for data sample with covariance matrix

We could also have created a dataset with a full covariance matrix using 

In [58]:
means= np.array([0.7579, 0.273537])
cov = np.array([0.1,0.01,0.01,0.1])
data_cov = astroabc.Model("normal",1000).make_mock(means,cov)

Keeping model simulation and distance methods the same as above. We can select a different way of estimating the covariance amongst all the particles using k-nearest neighbours. This returns a local covariance estimate and in many cases this reaches convergence faster then using a weighted covariance amongst all particles.

In [62]:
priors =  [('uniform', [0.1,0.9]), ('uniform', [0.1, 0.9])]
prop={'dfunc':dist_metric, 'outfile':"gaussian_example.txt", 'verbose':1, \
      'adapt_t': True, 'variance_method':4, 'k_near':10 }

In [63]:
sampler = astroabc.ABC_class(2,100,data_cov,[0.5,0.002],25,priors,**prop)

	 	
	 ########################     astroABC     ########################	
	 	
	 Npart=100 	 numt=25 	 tol=[0.5000,0.0020] exp
	 Priors= [('uniform', [0.1, 0.9]), ('uniform', [0.1, 0.9])]


In [64]:
sampler.sample(model_sim)

	 Running sampler...	 
	 Step: 0 	 tol: 0.5 	 Params: [0.6591580550676166, 0.36767196026619087]
	 Step: 1 	 tol: 0.402202418296 	 Params: [0.71371218530645275, 0.30987363467002216]
	 Step: 2 	 tol: 0.306427282137 	 Params: [0.71284492635210783, 0.31527459479897713]
	 Step: 3 	 tol: 0.2508977892 	 Params: [0.74473914993422785, 0.29838845335247927]
	 Step: 4 	 tol: 0.200445843669 	 Params: [0.73909365139990069, 0.26824707434162742]
	 Step: 5 	 tol: 0.171713356227 	 Params: [0.75298860019913183, 0.26886179788534714]
	 Step: 6 	 tol: 0.134273356043 	 Params: [0.75764783135403446, 0.28254474913316058]
	 Step: 7 	 tol: 0.115333409732 	 Params: [0.76673845296050291, 0.27657002002156095]
	 Step: 8 	 tol: 0.10438663762 	 Params: [0.769174873432943, 0.27324766031798597]
	 Step: 9 	 tol: 0.0878268438411 	 Params: [0.77023386880100175, 0.27553429491709658]
	 Step: 10 	 tol: 0.0650449594917 	 Params: [0.7721225530684489, 0.27218726392568543]
	 Step: 11 	 tol: 0.0575892424959 	 Params: [0.7758880927

# Running astroABC in parallel

To re-do the above analysis in parallel there are two options.
For running on a cluster it is best to use the mpi option and at run time specify the number of processors using

mpirun -np # 

To use the mpi option in astroABC simply set 'mpi': True in the keywords for the class. 

In [22]:
prop={'dfunc':dist_metric, 'outfile':"gaussian_example.txt", 'verbose':1, 'adapt_t': True, 'mpi': True}

You can then run the sample script in the examples folder to run the gaussian example on e.g. 16 processors using

$> mpirun -np 16 python gaussian.py gaussian_params.ini

For smaller jobs the Python multiprocessing option is available which can spawn multiple processes but which are still bound within a single node.
To use the mp option in astroABC simply set 'mp': True in the keywords for the class.
An additional option is to specify the number of threads using 'num_proc':number of threads for mp setting.
If none is specified then all available threads are used.

In [24]:
#to run on 4 threads
prop={'dfunc':dist_metric, 'outfile':"gaussian_example.txt", 'verbose':1, 'adapt_t': True, 'mp': True, 'num_proc':4}